# Infer SOMA - part 2

--------------

## OBJECTIVE:  Infer sub-cellular component SOMA in order to understand interactome 

To measure shape, position, and size of the cell body -- the soma.    There are a variety of signals from which we could make this inference.  The two most promising are a composite signal including the residual from linear unmixing (e.g. `ch = [1, 4, 5,7]`) and a signal derived from the lysosome channel (`ch = 1`).    In all procdures scaling the intensities to find the lower florescence signals at the edge of the soma against baseline is employed.  

In the long term we can build of a database of "ground truth" by sourcing additional markers which can be iteratively improved.  For example using the Allen Cell "Label Free" segmentation results should provide a good corroboration or constraints to the procedures outlined below.  

Three possible _workflows_ are illustrated below.

Dependencies:
The CYTOSOL inference rely on the NUCLEI AND SOMA inference.  Therefore all of the sub-cellular objects rely on this segmentation.





# IMPORTS

In [19]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np
import scipy

# # function for core algorithm
from scipy import ndimage as ndi
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage import morphology
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.utils.file_io import read_input_image, list_image_files, export_ome_tiff
from infer_subc.utils.img import *

%load_ext autoreload
%autoreload 2

from infer_subc.organelles.soma import infer_SOMA1, infer_SOMA2, infer_SOMA3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


--------------

# IMAGE PROCESSING Objective 2:  infer SOMA
  [OUTLINE: Objective #2](./00_pipeline_setup.ipynb)


## summary of steps (Workflow #1 & #2)
INPUT
- multiple channels in combination

PRE-PROCESSING
-  smoothe / remove noise

CORE-PROCESSING
-  segment objects
-  label / watershed

POST-PROCESSING
  - filter objects - fill holes, remove small

POST POST-PROCESS
  - Select single "most intense" soma
    - identify nucli which seeded the label
    - re-segment / label only that soma

OUTPUT
- object SOMA 






------------------------
# LOAD RAW IMAGE DATA
Identify path to _raw_ image data and load our example image


In [2]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)
test_img_name = img_file_list[5]


In [3]:

bioim_image = read_input_image(test_img_name)
img_data = bioim_image.image
raw_meta_data = bioim_image.raw_meta
ome_types = []
meta_dict = bioim_image.meta

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [18]:
chan_name = 'nuclei'
out_path = data_root_path / "inferred_objects" 
object_name = 'NU_object'

NU_bioim = read_input_image( out_path/ f"{object_name}.ome.tiff"  )
NU_object = NU_bioim.image
NU_labels = label(NU_object)


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


# WORKFLOW #1  - modified MCZ 3/20

Segmentation on a 3 channel composite as per 3/20 pipeline from MCZ
Summary - Starting with a linear combination of three signals,  the signal is smoothed and non-linearly combined (logrithmic and edge detected) for thresholding. 
## summary of steps

INPUT
- multi-channel sum (4*1,5,7)
- labeled NUCLEI (objective #1)

PRE-PROCESSING
- ne-noise and somoothe
- log transform inensities
- scale to max 1.0
- create non-linear aggregate of log-intensity + scharr filtered 

CORE PROCESSING
- mask object segmentation at bottom

POST-PROCESSING
  - fill holes
  - remove small objects

POST-PROCESSING
  - keep only the "most intense" Soma


OUTPUT
- mask of SOMA


In [5]:
##########################################################################
# DEFAULT PARAMETERS:
#   note that these parameters are supposed to be fixed for the structure
#   and work well accross different datasets
# default_params = defaultdict(str)

default_params = defaultdict(str, **{
    #"intensity_norm_param" : [0.5, 15]
    "intensity_norm_param" : [0],
    "gaussian_smoothing_sigma" : 1.34,
    "gaussian_smoothing_truncate_range" : 3.0,
    "dot_2d_sigma" : 2,
    "dot_2d_sigma_extra" : 1,
    "dot_2d_cutoff" : 0.025,
    "min_area" : 10,
    "low_level_min_size" :  100,
    "median_filter_size" : 10
})


################################

# calculate a filter dimension for median filtering which considers the difference in scale of Z
z_factor = scale[0]//scale[1]
med_filter_size = 4 #2D 
med_filter_size_3D = (1,med_filter_size,med_filter_size)  # set the scale for a typical median filter
print(f"median filtering scale is ~ : { [x*y for x,y in zip(scale,med_filter_size_3D)]}")

default_params['z_factor'] = z_factor
default_params['scale'] = scale



median filtering scale is ~ : [0.5804527163320905, 0.3194866073934927, 0.3194866073934927]


### INPUT

In [6]:

###################
# INPUT
###################
struct_img_raw = (4. * img_data[1,:,:,:].copy() + 
                               1. * img_data[5,:,:,:].copy() + 
                               1. * img_data[7,:,:,:].copy() )

# DEFAULT PARAMETERS:
#intensity_norm_param = [0.5, 15]
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100


med_filter_size = 15  


### PRE-PROCESSING

In [7]:
###################
# PRE_PROCESSING
###################
#

# Linear-ish smoothing
raw_soma_linear = intensity_normalization( struct_img_raw ,  scaling_param=scaling_param)

# structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
struct_img = median_filter_slice_by_slice( 
                                                                raw_soma_linear,
                                                                size=med_filter_size  )


structure_img_smooth = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                                        truncate_range=gaussian_smoothing_truncate_range,
                                                                                                                    )


# NON-Linear aggregation
log_image, d = log_transform( structure_img_smooth ) 
log_image = intensity_normalization(  log_image,  scaling_param=[0] )

edges = filters.scharr(log_image)

composite_soma = intensity_normalization(  edges,  scaling_param=[0] ) + log_image 


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


### CORE PROCESSING

In [8]:
###################
# CORE_PROCESSING
###################
    
# "Masked Object Thresholding" - 3D
bw, _bw_low_level = MO(composite_soma, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.5, 
                                                return_object=True,
                                                dilate=True)
                                                


### POST-PROCESSING

In [9]:
###################
# POST_PROCESSING
###################

# 2D 
width = 80  
removed_holes = aicssegmentation.core.utils.hole_filling(bw, hole_min =0. , hole_max=width**2, fill_2d = True) 

print(f" remove hole size  ~ : { scale[1]*width} microns, scale:{scale}")

width = 45  
cleaned_img = aicssegmentation.core.utils.size_filter(removed_holes, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= width**2, 
                                                         method = "slice_by_slice" ,
                                                         connectivity=1)
print(f" remove small objects  size  ~ : { scale[1]*width} microns, scale:{scale}")

# limit the labeling to where we have soma or NUclear signal
watershed_mask = np.logical_or(cleaned_img, NU_labels > 0)
inverted_img = 1. - composite_soma

labels_out = watershed(
            connectivity=np.ones((3, 3,3), bool),
            image=inverted_img,
            markers=NU_labels,
            mask=watershed_mask,
            )



 remove hole size  ~ : 6.3897321478698546 microns, scale:(0.5804527163320905, 0.07987165184837318, 0.07987165184837318)
 remove small objects  size  ~ : 3.594224333176793 microns, scale:(0.5804527163320905, 0.07987165184837318, 0.07987165184837318)


#### Visualize with `napari`
Visualize the first-pass segmentation and labeling with `napari`.

In [10]:

if viewer is None:
    viewer = napari.view_image(
        cleaned_img,
        scale=scale
    )
else: 
    viewer.add_image(
        cleaned_img,
        scale=scale
    )

viewer.scale_bar.visible = True
viewer.add_labels(
    labels_out,
    scale=scale 
)


<Labels layer 'labels_out' at 0x1475a2430>

### POST POST-PROCESSING

Re-mask the image based on the label which has the highst total intensity SOMA.  Then clean and re-label.

In [11]:
###################
# POST- POST_PROCESSING
###################

# keep the "SOMA" which contains the highest total signal
all_labels = np.unique(labels_out)[1:]

total_signal = [ raw_soma_linear[labels_out == label].sum() for label in all_labels]
# combine NU and "labels" to make a SOMA
keep_label = all_labels[np.argmax(total_signal)]

# now use all the NU labels which AREN't keep_label and add to mask and re-label
masked_composite_soma = composite_soma.copy()
new_NU_mask = np.logical_and( NU_labels !=0 ,NU_labels != keep_label)

masked_composite_soma[new_NU_mask] = 0

# "Masked Object Thresholding" - 3D
bw, _bw_low_level = MO(masked_composite_soma, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.5, 
                                                return_object=True,
                                                dilate=True)
                                                
# 2D Cleaning
width = 80  
removed_holes = aicssegmentation.core.utils.hole_filling(bw, hole_min =0. , hole_max=width**2, fill_2d = True) 
width = 45  
cleaned_img = aicssegmentation.core.utils.size_filter(removed_holes, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= width**2, 
                                                         method = "slice_by_slice" ,
                                                         connectivity=1)

watershed_mask = np.logical_or(cleaned_img, NU_labels == keep_label)
inverted_img = 1. - composite_soma

masked_labels_out = watershed(
            connectivity=np.ones((3, 3,3), bool),
            image=inverted_img,
            markers=NU_labels,
            mask=watershed_mask,
            )



In [12]:

viewer.add_labels(
    masked_labels_out,
    scale=scale 
)

# NOTE: for the test img there is an artifact in the upper left... might need to  first exclude regions touching the edge of the image 
# '/Users/ahenrie/Projects/Imaging/mcz_subcell/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.czi'

<Labels layer 'masked_labels_out' at 0x1696580a0>

## DEFINE `infer_SOMA1` function

In [2]:
##########################
# 2a.  infer_SOMA1
##########################
def _infer_SOMA1(struct_img: np.ndarray, NU_labels: np.ndarray,  in_params:dict) -> tuple:
    """
    Procedure to infer SOMA from linearly unmixed input.

    Parameters:
    ------------
    struct_img: np.ndarray
        a 3d image containing the SOMA signal

    NU_labels: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of SOMA
        label
            label (could be more than 1)
        parameters: dict
            updated parameters in case any needed were missing
    
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################                         

    #TODO: replace params below with the input params
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param

    # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    # Linear-ish processing
    med_filter_size = 15   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice(  struct_img,
                                                                            size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 
    gaussian_smoothing_sigma = 1.
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    # non-Linear processing
    log_img, d = log_transform( struct_img ) 
    log_img = intensity_normalization(  log_img,  scaling_param=[0] )

    struct_img = intensity_normalization(  filters.scharr(log_img),  scaling_param=[0] )  + log_img

    ###################
    # CORE_PROCESSING
    ###################
    local_adjust = 0.5
    low_level_min_size = 100
    # "Masked Object Thresholding" - 3D
    struct_obj, _bw_low_level = MO(struct_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)

    out_p["local_adjust"] = local_adjust 
    out_p["low_level_min_size"] = low_level_min_size 

    ###################
    # POST_PROCESSING
    ###################

    # 2D 
    hole_max = 80  
    struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    out_p['hole_max'] = hole_max

    small_object_max = 35
    struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max**3, 
                                                            method = "slice_by_slice" ,
                                                            connectivity=1)
    out_p['small_object_max'] = small_object_max

    labels_out = watershed(
                connectivity=np.ones((3, 3,3), bool),
                image=1. - struct_img,
                markers=NU_labels,
                mask= np.logical_or(struct_obj, NU_labels > 0),
                )
    ###################
    # POST- POST_PROCESSING
    ###################
    # keep the "SOMA" label which contains the highest total signal
    all_labels = np.unique(labels_out)[1:]

    total_signal = [ scaled_signal[labels_out == label].sum() for label in all_labels]
    # combine NU and "labels" to make a SOMA
    keep_label = all_labels[np.argmax(total_signal)]

    # now use all the NU labels which AREN't keep_label and add to mask and re-label
    masked_composite_soma = struct_img.copy()
    new_NU_mask = np.logical_and( NU_labels !=0 ,NU_labels != keep_label)

    # "Masked Object Thresholding" - 3D
    masked_composite_soma[new_NU_mask] = 0
    struct_obj, _bw_low_level = MO(masked_composite_soma, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)

    struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max**3, 
                                                            method = "slice_by_slice" ,
                                                            connectivity=1)
    masked_labels_out = watershed(
                connectivity=np.ones((3, 3,3), bool),
                image=1. - struct_img,
                markers=NU_labels,
                mask= np.logical_or(struct_obj, NU_labels == keep_label),
                )
                

    retval = (struct_obj,  masked_labels_out, out_p)
    return retval




In [ ]:
##  set up files
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"
# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)

chan_name = 'nuclei'
out_path = data_root_path / "inferred_objects" 
object_name = 'NU_object'

 
default_params = defaultdict(str, **{
    #"intensity_norm_param" : [0.5, 15]
    "intensity_norm_param" : [0],
    "gaussian_smoothing_sigma" : 1.34,
    "gaussian_smoothing_truncate_range" : 3.0,
    "dot_2d_sigma" : 2,
    "dot_2d_sigma_extra" : 1,
    "dot_2d_cutoff" : 0.025,
    "min_area" : 10,
    "low_level_min_size" :  100,
    "median_filter_size" : 10
})


################################

# calculate a filter dimension for median filtering which considers the difference in scale of Z
z_factor = scale[0]//scale[1]
med_filter_size = 4 #2D 
med_filter_size_3D = (1,med_filter_size,med_filter_size)  # set the scale for a typical median filter
print(f"median filtering scale is ~ : { [x*y for x,y in zip(scale,med_filter_size_3D)]}")

default_params['z_factor'] = z_factor
default_params['scale'] = scale
# DEFAULT PARAMETERS:
#intensity_norm_param = [0.5, 15]
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100



i = 4
target_file = img_file_list[i]


bioim_image = read_input_image(target_file)
img_data = bioim_image.image
raw_meta_data = bioim_image.raw_meta
ome_types = []
meta_dict = bioim_image.meta

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

#-------------------


raw_nuclei = img_data[0,:,:,:].copy()
NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 


###################
# SOMA INPUT
###################
raw_soma = (4. * img_data[1,:,:,:].copy() + 
                            1. * img_data[5,:,:,:].copy() + 
                            1. * img_data[7,:,:,:].copy() )

struct_img = intensity_normalization( raw_soma.copy() ,  scaling_param=scaling_param)
SO_object, SO_label, out_p =  _infer_SOMA1(struct_img.copy(), NU_label, out_p) 

In [ ]:
viewer = napari.view_image(
    raw_nuclei,
    scale=scale,
    blending='additive'
)
viewer.scale_bar.visible = True

viewer.add_image(
    NU_object,
    scale=scale,
    colormap='blue', 
    blending='additive'
)

viewer.dims.ndisplay = 3
viewer.camera.angles = (-30, 25, 120)

viewer.add_image(
    raw_soma,
    scale=scale,
    blending='additive'
)

viewer.add_image(
    SO_object,
    scale=scale,
    blending='additive'
)



------------------
# WORKFLOW #2

Segmentation on a 4 channel composite as per 6/22 CellProfiler pipeline from MCZ
## summary of steps

INPUT
- channel  1,4,5, and 7
- labeled NUCLEI (objective #1)

PRE-PROCESSING
-  scale to max 1.0
- gaussian  Filter window 10

CORE-PROCESSING
  - watershed from NU, global threshold, minimum cross-entropy
  - threshold smoothing scale: 1 pixel
  - threshold correction factor: .5 (more lenient)
  - lower / upper bounds  (0,1)
  - log transformed thresholding
  - fill holes
    - discard objects on borde
    - fill holes

POST-PROCESSING
  - fill holes
  - remove small objects

POST-PROCESSING
  - keep only the "most intense" Soma


OUTPUT
- mask of SOMA
- mask of NU (contained by SOMA)

## INPUT

In [13]:
###################
# INPUT
###################

composite_channels = [1,4,5,7]

gaussian_smoothing_sigma = 8
gaussian_smoothing_truncate_range = 3.0

gaussian_smoothing_sigma_3D = [gaussian_smoothing_sigma*scale[2]/x  for x in scale]

aicssegmentation.core.pre_processing_utils.suggest_normalization_param(img_data[1,:,:,:]) #  [0.0, 8]
#truncate_intensity = raw_soma.mean()+raw_soma.std()*3
raw_soma_linear = intensity_normalization(  img_data[composite_channels,:,:,:].copy(), scaling_param=[0] ).sum(axis=0)
raw_soma_linear = intensity_normalization(  raw_soma_linear, scaling_param=[0] )



mean intensity of the stack: 641.5999045901829
the standard deviation of intensity of the stack: 2144.942904845627
0.9999 percentile of the stack intensity is: 49648.38039997965
minimum intensity of the stack: 0
maximum intensity of the stack: 65535
suggested upper range is 23.0, which is 49975.2867160396
suggested lower range is 0.0, which is 641.5999045901829
So, suggested parameter for normalization is [0.0, 23.0]
To further enhance the contrast: You may increase the first value (may loss some dim parts), or decrease the second value(may loss some texture in super bright regions)
To slightly reduce the contrast: You may decrease the first value, or increase the second value
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


## PRE-PROCESSING

In [ ]:
###################
# PRE_PROCESSING
###################
struct_img = raw_soma_linear.copy()

# 2D smoothing with gaussian filter
gaussian_smoothing_sigma = 3
gaussian_smoothing_truncate_range = 3.0

med_filter_size = 9
#structure_img_median = ndi.median_filter(struct_img,    size=med_filter_size  ) #3D
structure_img_median = median_filter_slice_by_slice( 
                                                                struct_img,
                                                                size=med_filter_size )

structure_img_smooth = image_smoothing_gaussian_slice_by_slice(   structure_img_median,
                                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                                        truncate_range=gaussian_smoothing_truncate_range,
                                                                                                                    )



## CORE PROCESSING

In [14]:

# ###############################
# PARAMETERS:
###########################################
# #intensity_norm_param = [0.5, 15]
intensity_norm_param = [0]
gaussian_smoothing_sigma = 5
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100
#structure_img_smooth = raw_gaussian_filter2D
# this is closer to the original 

########################
#  CORE PROCESSING
######################
# "Masked Object Thresholding" - 3D

bw, _bw_low_level = MO(structure_img_smooth, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.25, 
                                                return_object=True,
                                                dilate=True)
                                                



mean intensity of the stack: 0.022594163479725156
the standard deviation of intensity of the stack: 0.035176025031434015
0.9999 percentile of the stack intensity is: 0.562098829807297
minimum intensity of the stack: 6.00946325238002e-09
maximum intensity of the stack: 1.0
suggested upper range is 15.5, which is 0.5678225514669525
suggested lower range is 0.5, which is 0.0050061509640081485
So, suggested parameter for normalization is [0.5, 15.5]
To further enhance the contrast: You may increase the first value (may loss some dim parts), or decrease the second value(may loss some texture in super bright regions)
To slightly reduce the contrast: You may decrease the first value, or increase the second value
mean intensity of the stack: 0.02121846708738522
the standard deviation of intensity of the stack: 0.03109671695313
0.9999 percentile of the stack intensity is: 0.49197553144795597
minimum intensity of the stack: 0.005571961371672658
maximum intensity of the stack: 0.7053786193844717


## POST-PROCESSING

In [ ]:
###################
# POST_PROCESSING
###################
# Clean in 3D

width = 5  
# discount z direction
#segmented_soma = remove_small_objects(bw, min_size=width*width*1.5, connectivity=1, in_place=False)
removed_holes = morphology.remove_small_holes(bw, width ** 3 )

width = 6  
cleaned_img = aicssegmentation.core.utils.size_filter(removed_holes, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= width**3, 
                                                         method = "3D", #"slice_by_slice" 
                                                         connectivity=1)

labels_out = watershed(
        image=np.abs(ndi.sobel(structure_img_smooth)),
        markers=NU_labels,
        connectivity=np.ones((3, 3, 3), bool),
        mask= np.logical_or(cleaned_img, NU_labels > 0),#cleaned_img,#
    )



### Visualize with `napari`

In [15]:


viewer.add_image(
    structure_img_smooth,
    scale=scale 
)

viewer.add_image(
    cleaned_img,
    scale=scale
)
viewer.scale_bar.visible = True

viewer.add_labels(
    labels_out,
    scale=scale 
)



<Labels layer 'labels_out [1]' at 0x16c64d190>

## POST POST-PROCESSING

In [16]:
###################
# POST POST_PROCESSING
###################

# keep the "SOMA" which contains the highest total signal
all_labels = np.unique(labels_out)[1:]

total_signal = [ raw_soma_linear[labels_out == label].sum() for label in all_labels]
# combine NU and "labels" to make a SOMA
keep_label = all_labels[np.argmax(total_signal)]
keep_label, total_signal

# now use all the NU labels which AREN't keep_label and add to mask and re-label
masked_composite_soma = structure_img_smooth.copy()
new_NU_mask = np.logical_and( NU_labels !=0 ,NU_labels != keep_label)

masked_composite_soma[new_NU_mask] = 0

# "Masked Object Thresholding" - 3D
bw, _bw_low_level = MO(masked_composite_soma, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.25, 
                                                return_object=True,
                                                dilate=True)
                                                

# clean in 3D
width = 5  
removed_holes = morphology.remove_small_holes(bw, width ** 3 )

width = 6  
cleaned_img = aicssegmentation.core.utils.size_filter(removed_holes, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= width**3, 
                                                         method = "3D", #"slice_by_slice" 
                                                         connectivity=1)

labels_out = watershed(
        image=np.abs(ndi.sobel(structure_img_smooth)),
        markers=NU_labels,
        connectivity=np.ones((3, 3, 3), bool),
        mask=cleaned_img,
    )

watershed_mask = np.logical_or(cleaned_img, NU_labels == keep_label)
masked_labels_out = watershed(
            connectivity=np.ones((3, 3,3), bool),
            image=np.abs(ndi.sobel(structure_img_smooth)),
            markers=NU_labels,
            mask=watershed_mask,
            )




In [17]:

viewer.add_labels(
    masked_labels_out,
    scale=scale 
)


<Labels layer 'masked_labels_out [1]' at 0x16c74fe20>

# DEFINE `infer_SOMA2` function

In [20]:


##########################
# 2b.  infer_SOMA2
########################### copy this to base.py for easy import
def _infer_SOMA2(struct_img: np.ndarray, NU_labels: np.ndarray,  in_params:dict) -> tuple:
    """
    Procedure to infer SOMA from linearly unmixed input.

    Parameters:
    ------------
    struct_img: np.ndarray
        a 3d image containing the SOMA signal

    NU_labels: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of SOMA
        label
            label (could be more than 1)
        parameters: dict
            updated parameters in case any needed were missing
    
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################                         
    #TODO: replace params below with the input params
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param


    # 2D smoothing
    # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    med_filter_size = 9   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice( 
                                                                    struct_img,
                                                                    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 3.
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    #    edges = filters.scharr(struct_img)
    # struct_img, d = log_transform( struct_img ) 
    # struct_img = intensity_normalization(  struct_img,  scaling_param=[0] )
    ###################
    # CORE_PROCESSING
    ###################
    # "Masked Object Thresholding" - 3D
    local_adjust = 0.25
    low_level_min_size = 100
    struct_obj, _bw_low_level = MO(struct_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)
    out_p["local_adjust"] = local_adjust 
    out_p["low_level_min_size"] = low_level_min_size 

    ###################
    # POST_PROCESSING
    ###################
    # 3D cleaning

    hole_max = 80  
    # discount z direction
    struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    out_p['hole_max'] = hole_max

    small_object_max = 35
    struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max**3, 
                                                            method = "3D", #"slice_by_slice" 
                                                            connectivity=1)
    out_p['small_object_max'] = small_object_max

    labels_out = watershed(
                                                image=np.abs(ndi.sobel(struct_img)),
                                                markers=NU_labels,
                                                connectivity=np.ones((3, 3, 3), bool),
                                                mask= np.logical_or(struct_obj, NU_labels > 0),
                                                )

    ###################
    # POST- POST_PROCESSING
    ###################
    # keep the "SOMA" label which contains the highest total signal
    all_labels = np.unique(labels_out)[1:]

    total_signal = [ scaled_signal[labels_out == label].sum() for label in all_labels]
    # combine NU and "labels" to make a SOMA
    keep_label = all_labels[np.argmax(total_signal)]

    # now use all the NU labels which AREN't keep_label and add to mask and re-label
    masked_composite_soma = struct_img.copy()
    new_NU_mask = np.logical_and( NU_labels !=0 ,NU_labels != keep_label)

    # "Masked Object Thresholding" - 3D
    masked_composite_soma[new_NU_mask] = 0
    struct_obj, _bw_low_level = MO(masked_composite_soma, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)
    # 3D cleaning
    struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max**3, 
                                                            method = "3D", #"slice_by_slice" 
                                                            connectivity=1)
    masked_labels_out = watershed(
                connectivity=np.ones((3, 3,3), bool),
                image=np.abs(ndi.sobel(struct_img)),
                markers=NU_labels,
                mask= np.logical_or(struct_obj, NU_labels == keep_label),
                )
                

    retval = (struct_obj,  masked_labels_out, out_p)
    return retval

-----------------------

# WORKFLOW #3

Segmentation on the log-scaled Lysosome signal and aggressively filling holes.
## summary of steps

INPUT
- channel  1 LYSOSOMES
- labeled NUCLEI (objective #1)

PRE-PROCESSING
- ne-noise and somoothe
- log transform inensities
- scale to max 1.0
- create non-linear aggregate of log-intensity + scharr filtered 

CORE PROCESSING
- mask object segmentation at bottom

POST-PROCESSING
  - fill holes
  - remove small objects

POST-PROCESSING
  - keep only the "most intense" Soma


OUTPUT
- mask of SOMA


## INPUT

In [62]:
###################
# INPUT
###################

composite_channels = [1]
struct_img_raw = intensity_normalization(  img_data[1,:,:,:].copy(), scaling_param=[0] )



intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


## PRE-PROCESSING

In [ ]:

#####################
# PRE=PROCESSING
#####################
# 2D linearish
struct_img_smooth = struct_img_raw
med_filter_size = 3   
# structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
struct_img_smooth2 = median_filter_slice_by_slice( 
                                                                struct_img_smooth,
                                                                size=med_filter_size  )

gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
struct_img_smooth3 = image_smoothing_gaussian_slice_by_slice(   struct_img_smooth2,
                                                                                                    sigma=gaussian_smoothing_sigma,
                                                                                                    truncate_range = gaussian_smoothing_truncate_range
                                                                                                )

# non-linear scaling (log)
log_image, d = log_transform( struct_img_smooth3 ) 
composite_img = intensity_normalization( log_image + filters.scharr(log_image),  scaling_param=[0] )  


## CORE PROCESSING

In [63]:
###################
# CORE_PROCESSING
###################
# "Masked Object Thresholding" - 3D
local_adjust = 0.5
struct_obj, _bw_low_level = MO(composite_img, 
                                            global_thresh_method='ave', 
                                            object_minArea=low_level_min_size, 
                                            extra_criteria=True,
                                            local_adjust= local_adjust, 
                                            return_object=True,
                                            dilate=True)


# # this is not actually applied for this workflow,,,,
# threshold_correction_factor = 0.9
# thresh_min, thresh_max = 0.0000267,.2

# threshold = min( max(threshold_value_log*threshold_factor, thresh_min), thresh_max)
# out_p['threshold_factor'] = threshold_factor
# out_p['thresh_min'] = thresh_min
# out_p['thresh_max'] = thresh_max


###################
# POST_PROCESSING
###################
# 3D cleanibng
hole_max = 100  
# discount z direction
struct_obj = aicssegmentation.core.utils.hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 

small_object_max = 30
struct_obj = aicssegmentation.core.utils.size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                        min_size= small_object_max**3, 
                                                        method = "slice_by_slice" ,
                                                        connectivity=1)


#  TEST: label without the edges in the composite
# labels_out1b = watershed(
#             connectivity=np.ones((3, 3,3), bool),
#             image=np.abs(filters.sobel(log_image)),
#             markers=NU_labels,
#             mask= np.logical_or(struct_obj, NU_labels > 0),
#             )

labels_out = watershed(
            connectivity=np.ones((3, 3,3), bool),
            image=np.abs(filters.sobel(composite_img)),
            markers=NU_labels,
            mask= np.logical_or(struct_obj, NU_labels > 0),
            )




### Visualize with `napari`

In [64]:


viewer.add_image(
    composite_img,
    scale=scale 
)

viewer.add_image(
    struct_obj,
    scale=scale
)
viewer.scale_bar.visible = True

viewer.add_labels(
    labels_out,
    scale=scale 
)



<Labels layer 'labels_out [1]' at 0x16cafd730>

## POST POST-PROCESSING

In [65]:

###################
# POST POST PROCESSING
###################

# keep the "SOMA" which contains the highest total signal
all_labels = np.unique(labels_out)[1:]

total_signal = [ struct_img_raw[labels_out == label].sum() for label in all_labels]
# combine NU and "labels" to make a SOMA
keep_label = all_labels[np.argmax(total_signal)]
keep_label, total_signal


(5,
 [2619.450797330675,
  2859.1222553264897,
  2610.5356374915605,
  990.0794537419587,
  57165.53406597422,
  3592.53461512864,
  7857.904203931077,
  66.31651789238128,
  3.7428702220861])

In [71]:

# now use all the NU labels which AREN't keep_label and add to mask and re-label
# 
# 
masked_composite_soma = composite_img.copy()
new_NU_mask = np.logical_and( NU_labels !=0 ,NU_labels != keep_label)

masked_composite_soma[new_NU_mask] = 0

masked_markers = NU_labels.copy()
masked_markers[new_NU_mask] = 0
# "Masked Object Thresholding" - 3D
bw, _bw_low_level = MO(masked_composite_soma, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=False)
                                                
hole_max = 80  
# discount z direction
struct_obj = aicssegmentation.core.utils.hole_filling(bw, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 

small_object_max = 30
struct_obj = aicssegmentation.core.utils.size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                        min_size= small_object_max**3, 
                                                        method = "slice_by_slice" ,
                                                        connectivity=1)

masked_labels_out = watershed(
            connectivity=np.ones((3, 3,3), bool),
            image=np.abs(filters.sobel(composite_img)),
            markers=masked_markers,
            mask= np.logical_or(struct_obj, NU_labels == keep_label),
            )



## Doesn't really work... 

In [72]:

viewer.add_labels(
    masked_labels_out,
    scale=scale 
)


viewer.add_labels(
    masked_markers,
    scale=scale 
)


<Image layer 'bw [3]' at 0x17311dfa0>

# DEFINE `infer_SOMA3` function

In [28]:


##########################
# 2c.  infer_SOMA3
##########################
def _infer_SOMA3(struct_img, NU_labels,  in_params) -> tuple:
    """
    Procedure to infer SOMA from linearly unmixed input.

    Parameters:
    ------------
    struct_img: np.ndarray
        a 3d image containing the SOMA signal

    NU_labels: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of SOMA
        label
            label (could be more than 1)
        parameters: dict
            updated parameters in case any needed were missing
    
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################                         
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param

   # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    med_filter_size = 3   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice( 
                                                                    struct_img,
                                                                    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 1.
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    log_img, d = log_transform( struct_img ) 
    struct_img = intensity_normalization(  log_img + filters.scharr(log_img) ,  scaling_param=[0] )  


    ###################
    # CORE_PROCESSING
    ###################
    # "Masked Object Thresholding" - 3D
    local_adjust = 0.5
    low_level_min_size = 100
    struct_obj, _bw_low_level = MO(struct_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)
    out_p["local_adjust"] = local_adjust 
    out_p["low_level_min_size"] = low_level_min_size 
    ###################
    # POST_PROCESSING
    ###################
    # 2D cleaning
    hole_max = 100  
    # discount z direction
    struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    out_p['hole_max'] = hole_max

    small_object_max = 30
    struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max**3, 
                                                            method = "slice_by_slice" ,
                                                            connectivity=1)
    out_p['small_object_max'] = small_object_max

    labels_out = watershed(
                                                image=np.abs(ndi.sobel(struct_img)),  #either log_img or struct_img seem to work, but more spurious labeling to fix in post-post for struct_img
                                                markers=NU_labels,
                                                connectivity=np.ones((3, 3, 3), bool),
                                                mask= np.logical_or(struct_obj, NU_labels > 0),
                                                )

    ###################
    # POST- POST_PROCESSING
    ###################
    # keep the "SOMA" label which contains the highest total signal
    all_labels = np.unique(labels_out)[1:]

    total_signal = [ scaled_signal[labels_out == label].sum() for label in all_labels]
    # combine NU and "labels" to make a SOMA
    keep_label = all_labels[np.argmax(total_signal)]

    # now use all the NU labels which AREN't keep_label and add to mask and re-label
    masked_composite_soma = struct_img.copy()
    new_NU_mask = np.logical_and( NU_labels !=0 ,NU_labels != keep_label)

    # "Masked Object Thresholding" - 3D
    masked_composite_soma[new_NU_mask] = 0
    struct_obj, _bw_low_level = MO(masked_composite_soma, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)

    # 2D cleaning
    struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max**3, 
                                                            method = "slice_by_slice" ,
                                                            connectivity=1)
    masked_labels_out = watershed(
                connectivity=np.ones((3, 3,3), bool),
                image=np.abs(ndi.sobel(struct_img)),  #either log_img or struct_img seem to work, but more spurious labeling to fix in post-post for struct_img
                markers=NU_labels,
                mask= np.logical_or(struct_obj, NU_labels == keep_label),
                )
                
    retval = (struct_obj,  masked_labels_out, out_p)
    return retval


In [ ]:

if viewer is None:
    viewer = napari.view_image(
        composite_soma,
        scale=scale,
        blending='additive',
        colormap='magenta',
    )

else:
    viewer.add_image(
        composite_soma,
        scale=scale 
    )

viewer.add_image(
    log_image,
    scale=scale 
)


--------------------------

# TEST `infer_SOMA{1,2,3}` exported functions


##
`infer_SOMA1` procedure

In [15]:
##  set up files
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)
test_img_name = img_file_list[5]

bioim_image = read_input_image(test_img_name)
img_data = bioim_image.image
raw_meta_data = bioim_image.raw_meta
ome_types = []
meta_dict = bioim_image.meta

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

#-------------------


chan_name = 'nuclei'
out_path = data_root_path / "inferred_objects" 
object_name = 'NU_object'

#NU_object_filen = export_ome_tiff(NU_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)

NU_bioim = read_input_image( out_path/ f"{object_name}.ome.tiff"  )
NU_object = NU_bioim.image
NU_labels = label(NU_object)

###################
# INPUT
###################
struct_img_raw = (4. * img_data[1,:,:,:].copy() + 
                               1. * img_data[5,:,:,:].copy() + 
                               1. * img_data[7,:,:,:].copy() )

# DEFAULT PARAMETERS:
#intensity_norm_param = [0.5, 15]
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100

default_params = defaultdict(str, **{
    #"intensity_norm_param" : [0.5, 15]
    "intensity_norm_param" : [0],
    "gaussian_smoothing_sigma" : 1.34,
    "gaussian_smoothing_truncate_range" : 3.0,
    "dot_2d_sigma" : 2,
    "dot_2d_sigma_extra" : 1,
    "dot_2d_cutoff" : 0.025,
    "min_area" : 10,
    "low_level_min_size" :  100,
    "median_filter_size" : 10
})

struct_img = intensity_normalization( struct_img_raw ,  scaling_param=scaling_param)


SO_object, SO_label, out_p =  infer_SOMA1(struct_img.copy(), NU_labels, default_params) 
# TODO:  make export ome_tiff export:   XX_object, XX_label, XX_signal
#              also fix Path vs. str action for export wrapper

viewer2 = None

/Users/ahenrie/Projects/Imaging/mcz_subcell/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.czi
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [16]:
# save results to a tiff file...
#  might just want to save as NUMPY arrays for now... 
chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'SO_object'
label_name = 'SO_label'

SO_object_filen = export_ome_tiff(SO_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)
SO_object_filen = export_ome_tiff(SO_label, meta_dict, label_name, str(out_path)+"/", curr_chan=0)


['SO_object']
['SO_label']


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [17]:

if viewer2 is None:
    viewer2 = napari.view_image(
        SO_object,
        scale=scale
    )
else: 
    viewer2.add_image(
        SO_object,
        scale=scale
    )


viewer2.add_labels(
    SO_label,
    scale=scale
)

<Labels layer 'SO_label' at 0x171eb1130>


## `infer_SOMA2` 

In [18]:
chan_name = 'nuclei'
out_path = data_root_path / "inferred_objects" 
object_name = 'NU_object'

#NU_object_filen = export_ome_tiff(NU_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)

NU_bioim = read_input_image( out_path/ f"{object_name}.ome.tiff"  )
NU_object = NU_bioim.image

NU_labels = label(NU_object)


###################
# INPUT
###################

composite_channels = [1,4,5,7]
raw_soma_linear = intensity_normalization(  img_data[composite_channels,:,:,:].copy(), scaling_param=[0] ).sum(axis=0)
#struct_img = intensity_normalization(  raw_soma_linear, scaling_param=[0] )


SO_object, SO_label, out_p =  infer_SOMA2(raw_soma_linear.copy(), NU_labels, default_params) 
# TODO:  make export ome_tiff export:   XX_object, XX_label, XX_signal
#              also fix Path vs. str action for export wrapper
# possibly need to do some post-post-processing to make suer that there is only a single SO_Object?


chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'SO_object2'

SO_object_filen = export_ome_tiff(SO_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)

# test: does this export work?
object_name = 'SO_label2'
SO_label_filen = export_ome_tiff(SO_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)

/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
['SO_object2']
['SO_label2']


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [19]:

viewer2.add_image(
    raw_soma_linear,
    scale=scale 
)

viewer2.add_image(
    SO_object,
    scale=scale
)
viewer2.scale_bar.visible = True

viewer2.add_labels(
    SO_label,
    scale=scale 
)



<Labels layer 'SO_label [1]' at 0x173a65fa0>



## `infer_SOMA3` 

In [20]:
chan_name = 'nuclei'
out_path = data_root_path / "inferred_objects" 
object_name = 'NU_object'

#NU_object_filen = export_ome_tiff(NU_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)

NU_bioim = read_input_image( out_path/ f"{object_name}.ome.tiff"  )
NU_object = NU_bioim.image

NU_labels = label(NU_object)

###################
# INPUT
###################
composite_channels = [1]
raw_soma_linear = intensity_normalization(  img_data[1,:,:,:].copy(), scaling_param=[0] )

#####################

SO_object, SO_label, out_p =  infer_SOMA3(raw_soma_linear.copy(), NU_labels, default_params) 
# TODO:  make export ome_tiff export:   XX_object, XX_label, XX_signal
#              also fix Path vs. str action for export wrapper
# possibly need to do some post-post-processing to make suer that there is only a single SO_Object?


chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'SO_object3'

SO_object_filen = export_ome_tiff(SO_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)

# test: does this export work?
object_name = 'SO_label3'
SO_label_filen = export_ome_tiff(SO_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)

/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
['SO_object3']


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


['SO_label3']


In [21]:
SO_object_filen

'/Users/ahenrie/Projects/Imaging/mcz_subcell/data/inferred_objects/SO_object3.ome.tiff'

In [22]:

viewer2.add_image(
    raw_soma_linear,
    scale=scale 
)

viewer2.add_image(
    SO_object,
    scale=scale
)
viewer2.scale_bar.visible = True

viewer2.add_labels(
    SO_label,
    scale=scale 
)

SO_object_filen

'/Users/ahenrie/Projects/Imaging/mcz_subcell/data/inferred_objects/SO_object3.ome.tiff'

In [ ]:
# adaptive_window = 200
# if adaptive_window % 2 == 0:
#     adaptive_window += 1
# local_threshold = filters.threshold_sauvola(
#     log_image, window_size=adaptive_window
# )


# # this implimentation doesn't seem to be working despite the fact that I've borrowed from 

# image_data = log_image
# volumetric = True
 
# tolerance=max(np.min(np.diff(np.unique(image_data))) / 2, 0.5 / 65536)
# tolerance=np.min(np.diff(np.unique(image_data))) / 2
# tolerance = None
# #th_method = "Li" #skimage.filters.threshold_li,
# window_size = 200
# th_method=filters.threshold_li
    
# local_threshold = cp_adaptive_threshold( image_data,
#                                                                     th_method, #skimage.filters.threshold_li,
#                                                                     volumetric,
#                                                                     window_size, 
#                                                                     tolerance
#                                                             )

# threshold_correction_factor = 0.9
# threshold_global = filters.threshold_li(image_data)
# corrected_threshold = local_threshold.copy()*threshold_correction_factor

# thresh_min, thresh_max = 0.0000267,.2


# # Constrain the local threshold to be within [0.7, 1.5] * global_threshold. It's for the pretty common case
# # where you have regions of the image with no cells whatsoever that are as large as whatever window you're
# # using. Without a lower bound, you start having crazy threshold s that detect noise blobs. And same for
# # very crowded areas where there is zero background in the window. You want the foreground to be all
# # detected.
# t_min = max(thresh_min, threshold_global * 0.7)
# t_max = min(thresh_max, threshold_global * 1.5)

# corrected_threshold[corrected_threshold < t_min] = t_min
# corrected_threshold[corrected_threshold > t_max] = t_max


# bw_adapt = image_data >= corrected_threshold
